## Supabase Uploader

In [32]:
import os
import pandas as pd
from supabase import create_client, Client
from dotenv import load_dotenv

load_dotenv()

url: str = os.environ.get("SUPABASE_URL")
key: str = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(url, key)

print("Supabase client created:", supabase)

Supabase client created: <supabase._sync.client.SyncClient object at 0x1193f41a0>


In [33]:
df = pd.read_json('../products/products.jsonl', lines=True)
print("DataFrame loaded:")
df.head()

DataFrame loaded:


,name,category,description,ingredients,price,rating,image_path
0,Cappuccino,Coffee,A rich and creamy cappuccino made with freshly...,"[Espresso, Steamed Milk, Milk Foam]",4.50,4.7,cappuccino_01.jpg
1,Jumbo Savory Scone,Bakery,"Deliciously flaky and buttery, this jumbo savo...","[Flour, Butter, Cheese, Herbs, Baking Powder, ...",3.25,4.3,savory_scone_01.jpg
2,Latte,Coffee,"Smooth and creamy, our latte combines rich esp...","[Espresso, Steamed Milk, Milk Foam]",4.75,4.8,latte_01.jpg
3,Chocolate Chip Biscotti,Bakery,"Crunchy and delightful, this chocolate chip bi...","[Flour, Sugar, Chocolate Chips, Eggs, Almonds,...",2.50,4.6,choco_chip_biscotti_01.jpg
4,Espresso shot,Coffee,"A bold shot of rich espresso, our espresso is ...",[Espresso],2.00,4.9,espresso_shot_01.jpg


In [34]:
# Define the image upload function
def upload_image(image_path):
    file_name = image_path.split('/')[-1]
    print(f"    Attempting to open: {image_path}")
    # Use the full image_path provided to the function
    with open(image_path, "rb") as f:
        imageUploadResponse = (
            supabase.storage
            # Use the bucket name (string) instead of the bucket object
            .from_("coffee-shop") 
            .upload(
                file=f,
                path=f"product_images/{file_name}",
                # Set upsert=True to allow overwriting if file exists
                file_options={"cache-control": "3600", "upsert": "true", "content-type":"image/jpeg"} 
            )
        )
    # Basic check (might need refinement based on actual response structure)
    # print(f"    Raw upload response: {imageUploadResponse}") # Debugging
    # Add more robust error checking based on Supabase library specifics if needed
    return imageUploadResponse

In [35]:
# Main processing loop
import os # Ensure os is imported in this cell if run independently

image_folder_path = '../products/images' # Relative to the notebook location

for index, row in df.iterrows():
    print(f"Processing index {index}: {row['name']}")
    
    # Construct the full path to the image file
    image_path = os.path.join(image_folder_path, row['image_path'])
    
    if not os.path.exists(image_path):
        print(f"  WARNING: Image not found at {image_path}, skipping upload.")
        continue # Skip if image doesn't exist

    try:
        # Upload the image
        print(f"  Uploading {image_path}...")
        upload_response = upload_image(image_path) 
        # Add check here based on actual response if needed
        print(f"  Upload completed.") # Simple confirmation

        # Construct the storage path used during upload
        file_name = image_path.split('/')[-1]
        storage_path = f"product_images/{file_name}"
        
        # Get the public URL for the uploaded image
        public_url = supabase.storage.from_("coffee-shop").get_public_url(storage_path)
        print(f"  Public URL: {public_url}")

        # Prepare data for insertion
        product_data = row.to_dict()
        product_data.pop('image_path') # Remove the local path
        product_data['image_url'] = public_url # Add the public Supabase URL
        
        # Insert data into the table
        print(f"  Inserting data for {row['name']}...")
        insertResponse = (
            supabase.table("products")
            .insert(product_data)
            .execute()
        )
        # Check insert response
        if hasattr(insertResponse, 'data') and insertResponse.data:
             print(f"  Insert successful: {len(insertResponse.data)} record(s)")
        elif hasattr(insertResponse, 'error') and insertResponse.error:
             print(f"  Insert FAILED: {insertResponse.error}")
        else:
             print(f"  Insert response: {insertResponse}") # Log other responses

    except Exception as e:
        print(f"  ERROR processing index {index} ({row['name']}): {e}")

print("Processing complete.")

Processing index 0: Cappuccino
  Uploading ../products/images/cappuccino_01.jpg...
    Attempting to open: ../products/images/cappuccino_01.jpg
  Upload completed.
  Public URL: https://fnpniwyitieenkemqkia.supabase.co/storage/v1/object/public/coffee-shop/product_images/cappuccino_01.jpg?
  Inserting data for Cappuccino...
  Insert successful: 1 record(s)
Processing index 1: Jumbo Savory Scone
  Uploading ../products/images/savory_scone_01.jpg...
    Attempting to open: ../products/images/savory_scone_01.jpg
  Upload completed.
  Public URL: https://fnpniwyitieenkemqkia.supabase.co/storage/v1/object/public/coffee-shop/product_images/savory_scone_01.jpg?
  Inserting data for Jumbo Savory Scone...
  Insert successful: 1 record(s)
Processing index 2: Latte
  Uploading ../products/images/latte_01.jpg...
    Attempting to open: ../products/images/latte_01.jpg
  Upload completed.
  Public URL: https://fnpniwyitieenkemqkia.supabase.co/storage/v1/object/public/coffee-shop/product_images/latte_